In [48]:
import json
import requests

In [84]:
id2imgurl = {}
asin2trecid = {}
trecid2asin = {}
doc_id = 0
collection = {}
with open('esci.json','r') as f:
    for l in f:
        j = json.loads(l)
        trecid2asin [doc_id] = j['asin']
        asin2trecid[j['asin']] = doc_id
        try:
            id2imgurl[doc_id] = j['image']
        except:
            pass
        j['doc_id'] = doc_id
        collection[doc_id] = j
        if 'description' not in collection[doc_id]:
            collection[doc_id]['description'] = ''
        if 'title' not in collection[doc_id]:
            collection[doc_id]['title'] = ''
        collection[doc_id]['docid'] = collection[doc_id]['doc_id']
        collection[doc_id]['text'] = collection[doc_id]['description']
        doc_id += 1

In [85]:
collection[0]

{'type': 'product',
 'locale': 'es',
 'asin': 'B06XXK1JQN',
 'title': 'Estores Basic- Enrollable Traslúcido , Crudo, 150x250 cm',
 'stars': '3,9 de 5 estrellas',
 'ratings': '90 valoraciones',
 'category': ['Hogar y cocina',
  'Decoración del hogar',
  'Decoración de ventanas',
  'Persianas',
  'Persianas enrollables y estores'],
 'attrs': {'Número de productos': '1',
  'Material': 'Textura',
  'Función especial': 'sin cuerda',
  'Marca': 'Estores Basic',
  'Color': 'Crudo',
  'Tamaño': '150x6x250 cm',
  'Peso del producto': '2.76 Libras',
  'Tipo de montaje': 'Montaje interior'},
 'bullets': ['Estor enrollable Traslúcido con contrapeso de PVC, oculto',
  'Mecanismo de accionamiento de cadeneta',
  'Soportes metálicos con embellecedores para instalación a techo o pared',
  'Limpieza con un paño ligeramente húmedo',
  'Dimensiones del producto \u200f : \u200e 150 x 6 x 250 cm; 1.25 kilogramos',
  'Producto en Amazon.es desde \u200f : \u200e 28 marzo 2017',
  'Fabricante \u200f : \u200e 

In [86]:
print("There are {} documents in the collection".format(doc_id -1))
print("There are {} img urls in the collection".format(len(id2imgurl)))

There are 1661907 documents in the collection
There are 1480714 img urls in the collection


In [87]:
with open('collection-imgs.json','w') as w:
    for i in id2imgurl:
        j = {'doc_id': i, 'image_url':id2imgurl[i]}
        w.write('{}\n'.format(json.dumps(j)))

In [88]:
asin2trecid[collection[0]['asin']]

0

In [89]:
collection[doc_id-1]

{'type': 'product',
 'locale': 'us',
 'asin': 'B002VUAM46',
 'title': 'Degree Men Antiperspirant Deodorant Adventure Antiperspirant For Men With MotionSense Technology, 2.7 Ounce (Pack of 4)',
 'stars': '4.7 out of 5 stars',
 'ratings': '2,256 ratings',
 'category': ['Beauty & Personal Care',
  'Personal Care',
  'Deodorants & Antiperspirants',
  'Antiperspirants'],
 'attrs': {'Material Type Free': 'ERROR: #N/A',
  'Item Form': 'Sticks',
  'Brand': 'DEGREE',
  'Item Volume': '2.7 Fluid Ounces',
  'Scent': 'Woodsy'},
 'bullets': ['We are upgrading! For a limited time, you may receive our old 48-hour Antiperspirant Deodorant while all our new 72-hour products make it to stores. Detailed changes in product description',
  'MOTIONSENSE TECHNOLOGY: This clinically proven antiperspirant provides a unique motion-activated, odor-control technology with 2x stronger sweat protection than regular antiperspirants',
  'READY FOR WHAT’S NEXT: Take on the day with Degree Men Adventure Antiperspirant 

# Generate a complete dump of all dataset withouth any language specific filtering

In [90]:
with open('full-collection.jsonl','w') as w:
    for i in range(0,len(collection)):
        doc_id = asin2trecid[collection[i]['asin']]
        collection[i]['doc_id'] = doc_id
        w.write("{}\n".format(json.dumps(j)))

## Generate a TREC collection style 

In [91]:
#doc_id\ttitle\tdescription
with open('collection.trec', 'w') as w:
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            w.write("{}\t{}\t{}\n".format(collection[i]['doc_id'],collection[i]['title'],collection[i]['description']))

## Generate a JSON collection style 

In [93]:
#json 
with open('corpus.jsonl', 'w') as w:
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            w.write("{}\n".format(json.dumps(collection[i])))

## Pyserini Compatable

Next we go ahead and make a collection in the trec format. 

In [94]:
# pyserini with just title and description
with open('pyserini-collection.jsonl') as w: 
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            j['id'] = collection[i]['doc_id']
            j['contents'] = json.dumps{'title':collection[i]['title'],'text':collection[i]['text']}
            w.write("{}\n".format(json.dumps(collection[i]))

SyntaxError: invalid syntax (889133158.py, line 6)

In [ ]:
! mkdir indexes

In [ ]:
! ls

In [ ]:
python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input tests/resources/sample_collection_jsonl \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

In [ ]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/sample_collection_jsonl')
hits = searcher.search('document')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

In [ ]:
# pyserini full
with open('pyserini-collection-full.jsonl') as w: 
    for i in range(0,len(collection)):
        if collection[i]['locale'] == 'us':
            j['id'] = collection[i]['doc_id']
            j['contents'] = json.dumps{collection[i]}
            w.write("{}\n".format(json.dumps(collection[i]))

In [ ]:
python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input tests/resources/sample_collection_jsonl \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

In [ ]:
Train	msmarco-doctrain-queries.tsv	15 MB	367,013	tsv: qid, query

In [ ]:
TREC submission: qid, “Q0”, docid, rank, score, runstring